In [1]:
try:
    import os
    import sys
    import elasticsearch
    from elasticsearch import Elasticsearch
    from elasticsearch import helpers
    import csv
    import requests
    import re
    import json
    from datetime import datetime
    from pprint import pprint
    import timeit
    import pandas as pd
    import numpy as np

    
    print("All modules are loaded")
except Exception as e:
    print("Some module are missing".format(e))

class SetEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.int64):
            return np.int(obj)
        # else
        return json.JSONEncoder.default(self, obj)

All modules are loaded


In [2]:
raw = pd.read_csv("dtksjan20_pupr.csv", header=0)
df = pd.DataFrame(raw)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1036451 entries, 0 to 1036450
Data columns (total 22 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   nmprop                1036451 non-null  object 
 1   nmkab                 1036451 non-null  object 
 2   nmkec                 1036426 non-null  object 
 3   nmdesa                1036303 non-null  object 
 4   pekerjaan_utama       1036451 non-null  object 
 5   kepemilikan_rumah     1036451 non-null  object 
 6   luas_lantai           1036451 non-null  int64  
 7   Jumlah_ART            1036451 non-null  int64  
 8   Stdt_luas_rmh         1036451 non-null  object 
 9   lantai_terluas        1036451 non-null  object 
 10  dinding_terluas       1036451 non-null  object 
 11  atap_terluas          1036451 non-null  object 
 12  Stdt_bangunan_rumah   1036451 non-null  object 
 13  sumber_airminum       1036451 non-null  object 
 14  Stdt_sumber_airminum  1036451 non-

In [3]:
df['id'] = np.random.randint(0,1036452, df.shape[0])

In [4]:
df.head()

,nmprop,nmkab,nmkec,nmdesa,pekerjaan_utama,kepemilikan_rumah,luas_lantai,Jumlah_ART,Stdt_luas_rmh,lantai_terluas,...,sumber_airminum,Stdt_sumber_airminum,fasilitas_bab,jenis_kloset,pembuangan_tinja,Stdt_sanitasi,rumah_layak_huni,kategori,Jumlah_bansos_rutin,id
0,ACEH,SIMEULUE,TEUPAH TENGAH,BUSUNG INDAH,Lainnya,Milik sendiri,77,3,Layak,Semen/bata merah,...,Air isi ulang,Layak,Tidak ada,Tidak pakai,Kolam/sawah/sungai/danau/laut,Tidak Layak,Tidak layak,Mampu,0.0,836870
1,ACEH,SIMEULUE,TEUPAH TENGAH,BUSUNG INDAH,Lainnya,Milik sendiri,36,1,Layak,Semen/bata merah,...,Sumur terlindung,Layak,Sendiri,Leher angsa,Tangki,Layak,Layak,Mampu,1.0,717054
2,ACEH,SIMEULUE,TEUPAH TENGAH,BUSUNG INDAH,Hotel & rumah makan,Milik sendiri,49,9,Tidak Layak,Semen/bata merah,...,Air isi ulang,Layak,Sendiri,Leher angsa,Tangki,Layak,Tidak layak,Mampu,0.0,664033
3,ACEH,SIMEULUE,TEUPAH TENGAH,BUSUNG INDAH,Jasa kemasyarakatan,Lainnya,30,2,Layak,Semen/bata merah,...,Air isi ulang,Layak,Tidak ada,Tidak pakai,Pantai/tanah lapang/kebun,Tidak Layak,Tidak layak,Mampu,1.0,460793
4,ACEH,SIMEULUE,TEUPAH TENGAH,BUSUNG INDAH,Jasa kemasyarakatan,Milik sendiri,25,3,Layak,Semen/bata merah,...,Air isi ulang,Layak,Tidak ada,Tidak pakai,Lainnya,Tidak Layak,Tidak layak,Mampu,1.0,611329


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1036451 entries, 0 to 1036450
Data columns (total 23 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   nmprop                1036451 non-null  object 
 1   nmkab                 1036451 non-null  object 
 2   nmkec                 1036426 non-null  object 
 3   nmdesa                1036303 non-null  object 
 4   pekerjaan_utama       1036451 non-null  object 
 5   kepemilikan_rumah     1036451 non-null  object 
 6   luas_lantai           1036451 non-null  int64  
 7   Jumlah_ART            1036451 non-null  int64  
 8   Stdt_luas_rmh         1036451 non-null  object 
 9   lantai_terluas        1036451 non-null  object 
 10  dinding_terluas       1036451 non-null  object 
 11  atap_terluas          1036451 non-null  object 
 12  Stdt_bangunan_rumah   1036451 non-null  object 
 13  sumber_airminum       1036451 non-null  object 
 14  Stdt_sumber_airminum  1036451 non-

In [6]:
res = requests.get('http://10.17.5.101:9200/')
print(res.content)

b'{\n  "name" : "semantic101",\n  "cluster_name" : "semanticDB",\n  "cluster_uuid" : "DDJhcfKTTuiO7RBCXE6mZA",\n  "version" : {\n    "number" : "7.6.1",\n    "build_flavor" : "default",\n    "build_type" : "rpm",\n    "build_hash" : "aa751e09be0a5072e8570670309b1f12348f023b",\n    "build_date" : "2020-02-29T00:15:25.529771Z",\n    "build_snapshot" : false,\n    "lucene_version" : "8.4.0",\n    "minimum_wire_compatibility_version" : "6.8.0",\n    "minimum_index_compatibility_version" : "6.0.0-beta1"\n  },\n  "tagline" : "You Know, for Search"\n}\n'


In [7]:
es = Elasticsearch("http://10.17.5.101:9200", timeout=30, max_retries=10, retry_on_timeout=True)
es.ping()

True

In [8]:
df2 = df.to_dict('records')

In [9]:
df2[1036450]

{'nmprop': 'SUMATERA UTARA',
 'nmkab': 'PADANG LAWAS UTARA',
 'nmkec': 'DOLOK SIGOMPULON',
 'nmdesa': 'SIPOGAS A',
 'pekerjaan_utama': 'Lainnya',
 'kepemilikan_rumah': 'Milik sendiri',
 'luas_lantai': 30,
 'Jumlah_ART': 3,
 'Stdt_luas_rmh': 'Layak',
 'lantai_terluas': 'Tanah',
 'dinding_terluas': 'Kayu/papan',
 'atap_terluas': 'Seng',
 'Stdt_bangunan_rumah': 'Tidak Layak',
 'sumber_airminum': 'Air permukaan',
 'Stdt_sumber_airminum': 'Tidak Layak',
 'fasilitas_bab': 'Tidak ada',
 'jenis_kloset': 'Tidak pakai',
 'pembuangan_tinja': 'Kolam/sawah/sungai/danau/laut',
 'Stdt_sanitasi': 'Tidak Layak',
 'rumah_layak_huni': 'Tidak layak',
 'kategori': nan,
 'Jumlah_bansos_rutin': 1.0,
 'id': 127939}

In [10]:
index_name = 'dtksjan20_hashed'
if es.indices.exists(index_name):
    es.indices.delete(index_name)

In [11]:
def generator(df2):
    for c, line in enumerate(df2):
        yield{
    '_index': 'dtksjan20_hashed',
    '_type': '_doc',
    '_id':line.get("id", None),
    '_source': {
        'nmprop':line.get('nmprop', ""),
        'nmkab':line.get('nmkab', ""),
        'nmkec':line.get('nmkec', ""),
        'nmdesa':line.get('nmdesa', ""),
        'pekerjaan_utama':line.get('pekerjaan_utama', ""),
        'kepemilikan_rumah':line.get('kepemilikan_rumah', ""),
        'luas_lantai':line.get('luas_lantai', ""),
        'jumlah_art':line.get('Jumlah_ART', ""),
        'Stdt_luas_rumah':line.get('Stdt_luas_rumah', ""),
        'lantai_terluas':line.get('lantai_terluas', ""),
        'dinding_terluas':line.get('dinding_terluas', ""),
        'atap_terluas':line.get('atap_terluas', ""),\
        'Stdt_bangunan_rumah':line.get('Stdt_bangunan_rumah', ""),
        'sumber_airminum':line.get('sumber_airminum', ""),
        'Stdt_sumber_airminum':line.get('Stdt_sumber_airminum', ""),
        'fasilitas_bab':line.get('fasilitas_bab', ""),
        'jenis_kloset':line.get('jenis_kloset', ""),
        'pembuangan_tinja':line.get('pembuangan_tinja', ""),
        'Stdt_sanitasi':line.get('Stdt_sanitasi', ""),
        'rumah_layak_huni':line.get('rumah_layak_huni', ""),
        'kategori':line.get('kategori', ""),
        'jumlah_bansos_rutin':line.get('Jumlah_bansos_rutin', "")
    }
        }
    raise StopIteration

In [12]:
test = generator(df2)

In [13]:
next(test)

{'_index': 'dtksjan20_hashed',
 '_type': '_doc',
 '_id': 836870,
 '_source': {'nmprop': 'ACEH',
  'nmkab': 'SIMEULUE',
  'nmkec': 'TEUPAH TENGAH',
  'nmdesa': 'BUSUNG INDAH',
  'pekerjaan_utama': 'Lainnya',
  'kepemilikan_rumah': 'Milik sendiri',
  'luas_lantai': 77,
  'jumlah_art': 3,
  'Stdt_luas_rumah': '',
  'lantai_terluas': 'Semen/bata merah',
  'dinding_terluas': 'Kayu/papan',
  'atap_terluas': 'Seng',
  'Stdt_bangunan_rumah': 'Layak',
  'sumber_airminum': 'Air isi ulang',
  'Stdt_sumber_airminum': 'Layak',
  'fasilitas_bab': 'Tidak ada',
  'jenis_kloset': 'Tidak pakai',
  'pembuangan_tinja': 'Kolam/sawah/sungai/danau/laut',
  'Stdt_sanitasi': 'Tidak Layak',
  'rumah_layak_huni': 'Tidak layak',
  'kategori': 'Mampu',
  'jumlah_bansos_rutin': 0.0}}

In [14]:
mapping = {
   "settings":{
      "number_of_shards":1,
      "number_of_replicas":0
   },
   "mappings":{
      "properties":{
         "nmprop":{
            "type":"keyword"
         },
         "nmkab":{
            "type":"keyword"
         },
         "nmkec":{
            "type":"keyword"
         },
         "nmdesa":{
            "type":"keyword"
         },
         "pekerjaan_utama":{
            "type":"text"
         },
         "kepemilikan_rumah":{
            "type":"keyword"
         },
         "luas_lantai":{
            "type":"integer"
         },
         "Jumlah_ART":{
            "type":"integer"
         },
         "Stdt_luas_rumah":{
            "type":"text"
         },
         "lantai_terluas":{
            "type":"text"
         },
         "dinding_terluas":{
            "type":"text"
         },
         "atap_terluas":{
            "type":"text"
         },
         "Stdt_bangunan_rumah":{
            "type":"text"
         },
         "sumber_airminum":{
            "type":"text"
         },
         "Stdt_sumber_ariminum":{
            "type":"text"
         },
         "fasilitas_bab":{
            "type":"text"
         },
         "jenis_kloset":{
            "type":"text"
         },
         "pembuangan_tinja":{
            "type":"text"
         },
         "Stdt_sanitasi":{
            "type":"text"
         },
         "rumah_layak_huni":{
            "type":"text"
         },
         "kategori":{
            "type":"text"
         },
         "Jumlah_bansos_rutin":{
            "type":"integer"
         }
      }
   }
}

In [15]:
Indexname = ''
testing = es.indices.create(index='dtksjan20_hashed', body=mapping, ignore=[400,404])

In [16]:
testing

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'dtksjan20_hashed'}

In [ ]:
try:
    res = helpers.bulk(es,generator(df2))
    print("Working")
except Exception as e:
    pass